In [1]:
# !pip install -U flash-attn --no-build-isolation
from qwen_vl_utils import process_vision_info
from transformers import AutoProcessor, AutoTokenizer, Qwen2VLForConditionalGeneration


model_path = "Qwen/Qwen2-VL-2B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_path)
processor = AutoProcessor.from_pretrained(model_path)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_path,
    torch_dtype="auto",
    # attn_implementation="flash_attention_2",
    device_map="cuda",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [1]:
messages = [
    [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": "./tes.png",
                },
                {
                    "type": "text",
                    "text": "find search icon of the left sidebar of visual studio code and Generate the caption in English with grounding:",
                },
            ],
        },
    ],
]

# Preparation for inference
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(text=text, images=image_inputs, videos=video_inputs, padding=True, return_tensors="pt")
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]

print(generated_ids)

# Output
output_text = tokenizer.decode(generated_ids_trimmed[0], skip_special_tokens=False, clean_up_tokenization_spaces=False)
output_tokens = tokenizer.convert_ids_to_tokens(generated_ids_trimmed[0])
print(output_text)
# print(output_tokens)

# output_text
# In the image,<|object_ref_start|>a woman<|object_ref_end|><|box_start|>(456,387),(702,789)<|box_end|> is sitting on the beach with<|object_ref_start|> her dog<|object_ref_end|><|box_start|>(241,431),(568,883)<|box_end|>, engaging in a playful activity where the dog is giving her a high-five. The setting is serene, with the ocean in the background and the sun casting a warm glow over the scene. The woman appears to be enjoying a peaceful moment with her pet.<|im_end|>
# output_tokens
# ['In', 'Ġthe', 'Ġimage', ',', '<|object_ref_start|>', 'a', 'Ġwoman', '<|object_ref_end|>', '<|box_start|>', '(', '4', '5', '6', ',', '3', '8', '7', '),(', '7', '0', '2', ',', '7', '8', '9', ')', '<|box_end|>', 'Ġis', 'Ġsitting', 'Ġon', 'Ġthe', 'Ġbeach', 'Ġwith', '<|object_ref_start|>', 'Ġher', 'Ġdog', '<|object_ref_end|>', '<|box_start|>', '(', '2', '4', '1', ',', '4', '3', '1', '),(', '5', '6', '8', ',', '8', '8', '3', ')', '<|box_end|>', ',', 'Ġengaging', 'Ġin', 'Ġa', 'Ġplayful', 'Ġactivity', 'Ġwhere', 'Ġthe', 'Ġdog', 'Ġis', 'Ġgiving', 'Ġher', 'Ġa', 'Ġhigh', '-five', '.', 'ĠThe', 'Ġsetting', 'Ġis', 'Ġserene', ',', 'Ġwith', 'Ġthe', 'Ġocean', 'Ġin', 'Ġthe', 'Ġbackground', 'Ġand', 'Ġthe', 'Ġsun', 'Ġcasting', 'Ġa', 'Ġwarm', 'Ġglow', 'Ġover', 'Ġthe', 'Ġscene', '.', 'ĠThe', 'Ġwoman', 'Ġappears', 'Ġto', 'Ġbe', 'Ġenjoying', 'Ġa', 'Ġpeaceful', 'Ġmoment', 'Ġwith', 'Ġher', 'Ġpet', '.', '<|im_end|>']

NameError: name 'processor' is not defined